In [ ]:
from PIL import Image
import numpy as np
from scipy.signal import convolve2d
import os


# GAUSSIAN FUNC
def gaussian_kernel(size: int, sigma: float) -> np.array:
    ax = np.linspace(-(size // 2), size // 2, size)
    xx, yy = np.meshgrid(ax, ax)
    kernel = np.exp(-(xx**2 + yy**2) / (2. * sigma**2))
    return kernel / np.sum(kernel)


# LAPLACIAN FUNC
def laplacian_kernel() -> np.array:
    return np.array([
        [0, -1, 0],
        [-1, 4, -1],
        [0, -1, 0]
    ])


# GAUSSIAN FUNC
def gaussian_blur(input_path: str, parameters: dict, output_path: str):
    try:
        if not os.path.exists(input_path):
            return False, f"Input file not found: {input_path}"

        size = parameters.get("size", 3)
        sigma = parameters.get("sigma", 1)

        img_obj = Image.open(input_path).convert("RGB")
        img_arr = np.array(img_obj)
        kernel = gaussian_kernel(size, sigma)
        blurred_arr = np.zeros_like(img_arr)
        for c in range(3):
            blurred_arr[:, :, c] = convolve2d(
                img_arr[:, :, c], kernel, mode='same', boundary='symm'
            )

        Image.fromarray(blurred_arr.astype(np.uint8)).save(output_path)
        return True, None

    except Exception as e:
        return False, str(e)


# EDGE DETECTION FUNC
def edge_detection(input_path: str, output_path: str):
    try:
        if not os.path.exists(input_path):
            return False, f"Input file not found: {input_path}"
        img_obj = Image.open(input_path).convert("L")
        img_arr = np.array(img_obj)
        kernel = laplacian_kernel()
        edges = convolve2d(img_arr, kernel, mode='same', boundary='symm')
        edges = np.clip(edges, 0, 255).astype(np.uint8)
        Image.fromarray(edges).save(output_path)
        return True, None
    
    except Exception as e:
        return False, str(e)


# COLOR DETECTION FUNC
def color_detection(input_path: str, parameters: dict, output_path: str):
    try:
        if not os.path.exists(input_path):
            return False, f"Input file not found: {input_path}"

        target_colors = parameters.get("target_colors", [])
        highlight_colors = parameters.get("highlight_colors", [])
        if not target_colors:
            return False, "No target colors provided."

        img_obj = Image.open(input_path).convert("RGB")
        img_arr = np.array(img_obj)
        result = img_arr.copy()
        if len(highlight_colors) == 1:
            highlight_colors = highlight_colors * len(target_colors)
        elif len(highlight_colors) != len(target_colors):
            return False, "Highlight colors length mismatch with target colors."

        # Replace matching pixels
        for target, highlight in zip(target_colors, highlight_colors):
            mask = np.all(img_arr == target, axis=-1)
            result[mask] = highlight

        Image.fromarray(result.astype(np.uint8)).save(output_path)
        return True, None

    except Exception as e:
        return False, str(e)


# BINARY IMAGE 
def binary_image(input_path: str, parameters: dict, output_path: str):
    try:
        if not os.path.exists(input_path):
            return False, f"Input file not found: {input_path}"

        pivot = parameters.get("pivot", 0.5)
        if not (0 <= pivot <= 1):
            return False, "Pivot must be between 0 and 1."

        threshold = int(255 * pivot)
        img_obj = Image.open(input_path).convert("L")
        img_arr = np.array(img_obj)
        binary_arr = np.where(img_arr > threshold, 255, 0).astype(np.uint8)

        Image.fromarray(binary_arr).save(output_path)
        return True, None

    except Exception as e:
        return False, str(e)


# GRAYSCALE
def grayscale_tool(input_path: str, parameters: dict, output_path: str):
    try:
        if not os.path.exists(input_path):
            return False, f"Input file not found: {input_path}"

        method = parameters.get("method", "simple").lower()
        img_obj = Image.open(input_path).convert("RGB")
        img_arr = np.array(img_obj)

        arr = []
        for row in img_arr:
            _arr = []
            for val in row:
                r, g, b = val
                if method == "simple":
                    avg_val = int((r + g + b) / 3)
                elif method == "weighted":
                    avg_val = int(0.299 * r + 0.587 * g + 0.114 * b)
                else:
                    return False, f"Invalid Method: {method}"
                _arr.append(avg_val)
            arr.append(_arr)

        gray_arr = np.array(arr).astype(np.uint8)
        gray_rgb = np.stack((gray_arr, gray_arr, gray_arr), axis=2)
        Image.fromarray(gray_rgb).save(output_path)
        return True, None

    except Exception as e:
        return False, str(e)


# MAIN FUNC
def main():
    print("Image Processing Toolkit")
    input_path = r"data\image_1.png"
    output_path = r"data\outputs\outputs.png"

    print("Choose operation to execute:")
    print("1. Gaussian Blur")
    print("2. Edge Detection")
    print("3. Color Detection")
    print("4. Binary Image")
    print("5. Grayscale Tool")
    choice = input("Enter choice (1-5): ")

    if choice == "1":
        size = int(input("Enter blur size : "))
        sigma = float(input("Enter sigma : "))
        res, err = gaussian_blur(input_path, {"size": size, "sigma": sigma}, output_path)

    elif choice == "2":
        res, err = edge_detection(input_path, output_path)

    elif choice == "3":
        target_colors_str = input("Enter target colors (r,g,b) separated by semicolon: ")
        target_colors = [tuple(map(int, c.strip().split(','))) for c in target_colors_str.split(';')]
        highlight_color_str = input("Enter highlight color (r,g,b): ")
        highlight_colors = [tuple(map(int, highlight_color_str.strip().split(',')))]
        res, err = color_detection(input_path, {"target_colors": target_colors, "highlight_colors": highlight_colors}, output_path)

    elif choice == "4":
        pivot = float(input("Enter pivot value (0-1) for binary image: "))
        res, err = binary_image(input_path, {"pivot": pivot}, output_path)

    elif choice == "5":
        method = input("Enter grayscale method (simple/weighted): ")
        res, err = grayscale_tool(input_path, {"method": method}, output_path)

    else:
        print("Invalid choice.")
        return

    print("Processing Successful!" if res else f"Processing Failed: {err}")


if __name__ == "__main__":
    main()


Image Processing Toolkit
Choose operation to execute:
1. Gaussian Blur
2. Edge Detection
3. Color Detection
4. Binary Image
5. Grayscale Tool
Invalid choice.
